In [1]:

import time
import datetime
#Libary to send Emails to self
import smtplib

import csv
import pandas as pd
import os

import subprocess



In [6]:
import requests
import codecs
from urllib.parse import urlparse  # For filename extraction

site_to_check = [
                    #Fresh Produce
                    'https://www.amazon.com/Organic-Honeycrisp-Apple-One-Medium/dp/B001GIP2A8/ref=sr_1_3_f3_wg?almBrandId=QW1hem9uIEZyZXNo&fpw=alm&s=amazonfresh&sr=1-3',
                    'https://www.amazon.com/Dole-Organic-Bananas-Bag/dp/B07ZLF9G83?pd_rd_i=B07ZLF9G83&fpw=alm&almBrandId=QW1hem9uIEZyZXNo&ref_=pd_alm_fs_dsk_sf_ai_16318981_1_1_i',
                    'https://www.amazon.com/produce-aisle-Strawberries-1-lb/dp/B000P6J0SM?pd_rd_i=B000P6J0SM&fpw=alm&almBrandId=QW1hem9uIEZyZXNo&ref_=pd_alm_fs_dsk_sf_ai_16318981_1_3_i',
                    'https://www.amazon.com/Shrimp-White-Farm-Raised-Frozen/dp/B07FZFB494?pd_rd_i=B07FZFB494&fpw=alm&almBrandId=VUZHIFdob2xlIEZvb2Rz&ref_=pd_alm_wf_dsk_dp_dzrp_1_6_i',
                    'https://www.amazon.com/Seafood-Halibut-Fillet-Msc/dp/B07HMC7VGJ?pd_rd_i=B07HMC7VGJ&fpw=alm&almBrandId=VUZHIFdob2xlIEZvb2Rz&ref_=pd_alm_wf_dsk_dp_dzrp_1_2_i'
                     ]


def write_html_file(site, proxy=None, folder_path='downloaded_html', prettify=False):
    """
    Attempts to download the content of a website using a proxy and writes it to a file
    within a specified folder. Optionally, prettifies the downloaded HTML content.

    Args:
        site (str): The URL of the website to access.
        proxy (str, optional): The proxy address (e.g., 'http://127.0.0.1:8080'). Defaults to None.
        folder_path (str, optional): The path to the folder where HTML files will be saved. Defaults to 'downloaded_html'.
        prettify (bool, optional): Whether to prettify the downloaded HTML content before writing (improves readability). Defaults to False.
    """

    try:
        # Create the folder if it doesn't exist
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)

        # Extract filename between 2nd and 3rd backslashes (consider potential naming issues with special characters)
        url_parts = urlparse(site).path.split('/')
        if len(url_parts) >= 2:  # Ensure there are at least 2 parts (1 backslash)
            filename = f"{url_parts[1]}.html"
        else:
            filename = f"unknown_{site.split('/')[-1]}.html"  # Fallback for malformed URLs

        print(f'Downloading content for {site}...')

        headers = {'User-Agent':
                    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36',
                   'Cache-Control': 'no-cache'}  # Disable caching in headers

        response = requests.get(site, proxies={'http': proxy, 'https': proxy}, headers=headers)

        if response.status_code == 200:
            # Construct the full path to the file within the folder
            full_path = os.path.join(folder_path, filename)

            # Downloaded content
            html_content = response.text

            if prettify:
                # Prettify the HTML content using BeautifulSoup (optional)
                soup = BeautifulSoup(html_content, 'html.parser')
                pretty_html = soup.prettify()
                html_content = pretty_html
            else:
                # Keep the content as-is
                pass

            with codecs.open(full_path, 'w', encoding='utf-8') as f:
                f.write(html_content)
            print(f'{site} content written to {full_path}')
        else:
            print(f'Failed to download {site} (Status code: {response.status_code})')
    except requests.exceptions.RequestException as e:
        print(f'Error downloading {site}: {e}')
    

# Site List of What you want to scrape
if __name__ == '__main__':
    sites_to_check = site_to_check

    # Call the function for each website (optionally with a proxy)
    for site in site_to_check:
        write_html_file(site)  # Without proxy
        # write_html_file(site, proxy='http://your_proxy_address:port')  # With proxy


https://www.amazon.com/Organic-Honeycrisp-Apple-One-Medium/dp/B001GIP2A8/ref=sr_1_3_f3_wg?almBrandId=QW1hem9uIEZyZXNo&fpw=alm&s=amazonfresh&sr=1-3 content written to downloaded_html\Organic-Honeycrisp-Apple-One-Medium.html
https://www.amazon.com/Dole-Organic-Bananas-Bag/dp/B07ZLF9G83?pd_rd_i=B07ZLF9G83&fpw=alm&almBrandId=QW1hem9uIEZyZXNo&ref_=pd_alm_fs_dsk_sf_ai_16318981_1_1_i content written to downloaded_html\Dole-Organic-Bananas-Bag.html
https://www.amazon.com/produce-aisle-Strawberries-1-lb/dp/B000P6J0SM?pd_rd_i=B000P6J0SM&fpw=alm&almBrandId=QW1hem9uIEZyZXNo&ref_=pd_alm_fs_dsk_sf_ai_16318981_1_3_i content written to downloaded_html\produce-aisle-Strawberries-1-lb.html
https://www.amazon.com/Shrimp-White-Farm-Raised-Frozen/dp/B07FZFB494?pd_rd_i=B07FZFB494&fpw=alm&almBrandId=VUZHIFdob2xlIEZvb2Rz&ref_=pd_alm_wf_dsk_dp_dzrp_1_6_i content written to downloaded_html\Shrimp-White-Farm-Raised-Frozen.html
https://www.amazon.com/Seafood-Halibut-Fillet-Msc/dp/B07HMC7VGJ?pd_rd_i=B07HMC7VGJ&fpw

In [14]:
#The Final Product

import subprocess
from datetime import date  # Import date for today's date
from bs4 import BeautifulSoup  # Import BeautifulSoup for parsing HTML
import csv  # Import csv library for writing
import os

from proxy_rotation import write_html_file


def check_price():
    # Run the proxy rotation (assuming proxy_rotation.py is a separate script)
    subprocess.run(["python", "proxy_rotation.py"])

    # Path to the folder containing HTML files
    folder_path = "downloaded_html"

    counter = 0 #Link Counter
    # Iterate over files in the folder
    for filename in os.listdir(folder_path):
        
        
        if filename.endswith(".html"):  # Check if it's an HTML file
            full_path = os.path.join(folder_path, filename)

            try:
                # Open the file and process its contents
                with open(full_path, 'r', encoding='utf-8') as f:
                    html_content = f.read()

                    # Process the HTML content (extract price, etc.)
                    soup = BeautifulSoup(html_content, 'html.parser')

                    # Access and process the parsed HTML using BeautifulSoup methods
                    title = soup.find(id='productTitle').text.strip()
                    price = soup.find(class_='a-price-whole').text.strip() + soup.find(class_='a-price-fraction').text.strip()
                    today = date.today()
                    
                    link = site_to_check[counter] 
                    
                                       
                
                    print(f"Processing price data from {filename}")
                    

                    # ----------------------------------Writing into a csv ------------------------------------
                    header = ['Title', 'Price', 'Date', 'Link']
                    data = [[title, price, today, link]]

                    # Check if the CSV file exists
                    file_exists = os.path.isfile('amazon_web_scrapper_dataset.csv')

                    if not file_exists:  # Create the CSV file if it doesn't exist
                        with open('amazon_web_scrapper_dataset.csv', 'w', newline='', encoding='utf-8') as f:
                            writer = csv.writer(f)
                            writer.writerow(header)
                            writer.writerows(data)
                        print('CSV file created successfully!')
                    else:  # Append data if the CSV file already exists
                        with open('amazon_web_scrapper_dataset.csv', 'a', newline='', encoding='utf-8') as f:
                            writer = csv.writer(f)
                            writer.writerows(data)
                        print('Data appended to existing CSV file.')

            except Exception as e:  # Catch any exceptions during processing or writing
                print(f"Error processing {filename}: {e}")
    
        counter += 1 #Link Counter

if __name__ == "__main__":
    check_price()


Processing price data from Dole-Organic-Bananas-Bag.html
Data appended to existing CSV file.
Processing price data from Organic-Honeycrisp-Apple-One-Medium.html
Data appended to existing CSV file.
Processing price data from produce-aisle-Strawberries-1-lb.html
Data appended to existing CSV file.
Processing price data from Seafood-Halibut-Fillet-Msc.html
Data appended to existing CSV file.
Processing price data from Shrimp-White-Farm-Raised-Frozen.html
Data appended to existing CSV file.


In [ ]:
#-------------------------Automating the script into time intervals------------------------
while(True):
    check_price()
    time.sleep(10)

In [33]:
import os

def clear_csv(filename, keep_header=False):
  """
  Clears the contents of a CSV file, optionally keeping the header line.

  Args:
      filename (str): The path to the CSV file.
      keep_header (bool, optional): Whether to keep the header line (default: False).
  """

  try:
    if os.path.exists(filename):
      with open(filename, 'r', newline='') as f:
        lines = f.readlines()  # Read all lines into a list

      if keep_header:  # Check only if keeping the header
        lines = lines[2:]  # Skip the first line (header) if requested

      else:
        lines = []  # Empty list to clear the content

      with open(filename, 'w', newline='') as f:
        f.writelines(lines)  # Write the modified lines back to the file

      if keep_header:
        print(f'CSV file "{filename}" header preserved!')
      else:
        print(f'CSV file "{filename}" cleared successfully!')
    else:
      print(f'CSV file "{filename}" does not exist.')
  except Exception as e:
    print(f'Error clearing CSV file: {e}')

# Example usage (clear everything)
clear_csv('amazon_web_scrapper_dataset.csv')

# Example usage (keep the header)
clear_csv('amazon_web_scrapper_dataset.csv', keep_header=True)


CSV file "amazon_web_scrapper_dataset.csv" cleared successfully!
CSV file "amazon_web_scrapper_dataset.csv" header preserved!
